In [ ]:
from openai import OpenAI
import time

# Added key here
with open("api_key.txt", "r") as file:
    api_key = file.read()
client = OpenAI(
    api_key=api_key,
)

# Added the assistant here
my_assistant = client.beta.assistants.retrieve("asst_9DQgg0cSnpmvioyIayNCMKNs")

In [50]:
def main():

    # User input for number of questions
    num_of_questions = input('How many questions would you like to do today? Please input a number.')
    try:
        num_of_questions = int(num_of_questions)
    except:
        num_of_questions = 1

    # Create a thread
    thread = client.beta.threads.create()


    def generate_question(user_level):

        message = client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=f"Create a level {user_level} difficulty question related to LIGN 101. The level is from 1 to 5, where 1 is the easiest, and 5 is the hardest."
        )
        # 第二句话应该写到instructions里去，然后删掉

        run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=my_assistant.id
        )
        
        # Check status
        while True:
            time.sleep(3)
            run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
            )
            if run.status == 'completed':
                break
            elif run.status == 'failed':
                return 'FAILED'

        messages = client.beta.threads.messages.list(
        thread_id=thread.id
        )
        return messages.data[0].content[0].text.value


    def answer_question(answer):

        message = client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=f"My answer: {answer}\nPlease evaluate this answer with exactly one of these: 'Correct!', 'This is not accurate', or 'Sorry, this is wrong.' Then, you should state directly the correct answer to it."
        )
        # Please evaluate...这句话也应该写到instructions里去，然后删掉。最后一句可以留下。

        run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=my_assistant.id
        )
        
        # Check status
        while True:
            time.sleep(3)
            run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
            )
            if run.status == 'completed':
                break
            elif run.status == 'failed':
                return 'FAILED'

        messages = client.beta.threads.messages.list(
        thread_id=thread.id
        )
        return messages.data[0].content[0].text.value


    def check_if_wrong(text):
        words_to_check = ['incorrect', 'not correct', 'not accurate', 'inaccurate', 'wrong']
        for word in words_to_check:
            if word in text:
                return True
        return False


    # Initial user's level
    user_level = 3


    # Generate multiple questions consecutively
    for _ in range(num_of_questions):
        
        question = generate_question(user_level)
        print(question + '\n')

        user_input = input(question)
        if len(user_input) < 1:
            user_input = "Skip."
        print(user_input + '\n')
        
        evaluation = answer_question(user_input)
        print(evaluation + '\n')


        if check_if_wrong(evaluation.lower()):
            user_level = max(1, user_level - 1)
        elif 'correct' in evaluation.lower():
            user_level = min(5, user_level + 1)


    # After the last question
    
    # 总结一下对了几道错了几道
    # 回顾答错的题...
    # Want more questions?

    print(f'Congratulations! You have completed {num_of_questions} questions today. Good luck on your finals.')


In [51]:
main()

Sure! Here's a level 3 difficulty question related to LIGN 101:

Question: What is the difference between a phoneme and an allophone?

Please take your time to answer the question. Once you're ready, let me know your answer and I will provide feedback.

what is phoneme?

This is not accurate.

A phoneme is the smallest unit of sound in a language that can distinguish meaning. It is an abstract representation of the sounds that speakers perceive and produce. Phonemes are categorized based on their distinctiveness and phonological rules governing their usage in a particular language.

The correct answer is: 

A phoneme is the smallest unit of sound in a language that can distinguish meaning. It represents a group of sounds that are perceived as the same by native speakers.

Sure! Here's a level 2 difficulty question related to LIGN 101:

Question: What is the study of the different sounds in human languages called?

Please take your time to answer the question. Once you're ready, let me 

In [ ]:
# 这是什么？

# This function would send a user's response to a question to GPT-4 for evaluation.
def evaluate_answer(user_answer, question):
    prompt = f"Given the linguistics question '{question}', evaluate the following answer: {user_answer}"
    response = openai.Completion.create(
        engine="gpt-4",
        prompt=prompt,
        max_tokens=250
    )
    return response.choices[0].text.strip()

@app.route('/get_question', methods=['POST'])
def get_question():
    data = request.json
    user_level = data.get('user_level')
    question = generate_question(user_level)
    return jsonify(question=question)

@app.route('/submit_answer', methods=['POST'])
def submit_answer():
    data = request.json
    user_answer = data.get('user_answer')
    question = data.get('question')
    evaluation = evaluate_answer(user_answer, question)
    return jsonify(evaluation=evaluation)

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
# 下面的都能跑
# 能跑就不要乱改
# 不要乱改

In [152]:
def main():

    # User input for number of questions
    num_of_questions = input('How many questions would you like to do today? Please input a number.')
    try:
        num_of_questions = int(num_of_questions)
    except:
        num_of_questions = 1


    def generate_question(user_level):
        prompt = f"Create a level {user_level} difficulty question related to LIGN 101. The level is from 1 to 5, where 1 is the easiest, and 5 is the hardest."
        completion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "system", "content": prompt},
        ]
        )
        return completion.choices[0].message.content


    def answer_question(answer):
        prompt1 = f"{question}\nMy answer: {answer}"
        prompt2 = "Please evaluate this answer with exactly one of these: 'Correct!', 'This is not accurate', or 'Sorry, this is wrong.' Then, you should state directly the correct answer to it."
        completion = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=[
            {"role": "user", "content": prompt1},
            {"role": "user", "content": prompt2},
        ]
        )
        return completion.choices[0].message.content


    user_level = 3


    # Generate multiple questions consecutively
    for _ in range(num_of_questions):
        
        question = generate_question(user_level)
        print(question + '\n')

        user_input = input(question)
        if len(user_input) < 1:
            user_input = "Skip."
        print(user_input + '\n')
        
        evaluation = answer_question(user_input)
        print(evaluation + '\n')


        if 'this is wrong.' in evaluation:
            user_level = max(1, user_level - 1)
        elif 'Correct!' in evaluation:
            user_level = min(5, user_level + 1)


    # After the last question
    
    # 总结一下对了几道错了几道
    # 回顾答错的题...
    # Want more questions?

    print(f'Congratulations! You have completed {num_of_questions} questions today. Good luck on your finals.')


In [153]:
main()

What is the difference between phonetics and phonology in the context of linguistics? Provide examples to illustrate your answer.

e

Sorry, this is wrong. 
The correct answer is: Phonetics is the study of the physical production, transmission, and perception of sounds in speech, while phonology is the study of the sound patterns and the systematic organization of sounds within a particular language or languages. For example, in English, the phoneme /p/ is pronounced differently in the words "pat" and "spat," and phonology studies the patterns and rules governing these different pronunciations.

What is the difference between syntax and semantics in the context of natural language processing? (Level 2)

d

This is not accurate. 

The difference between syntax and semantics is that syntax refers to the structure and grammar of a language, including the arrangement of words and phrases, while semantics refers to the meaning and interpretation of words and sentences within a language. In 

In [ ]:
# IMPORTANT EXAMPLE


# CONNECT WITH ASSISTANT

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=my_assistant.id
)

# CHECK STATUS
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
run.status

# IF FAILED
run.last_error

# CHECK STATUS BEFORE RUN THIS

messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
print(messages.data[0].content[0].text.value)